# Image Segmentation

The algorithm consists of four steps:

### (i) re-segment the image,

### (ii) sample noise model parameters,

### (iii) sample MRF model parameters,

### (iv) sample the number of classes.

We will implement each of those sections below.

If you need to access the paper, go here:
https://static.aminer.org/pdf/PDF/000/180/822/unsupervised_image_segmentation_using_markov_random_field_models.pdf

In [1]:
import numpy as np

## Run the entire algorithm on an image.

In [2]:
def segment(img, numiter=100, alpha=(1.1, 10.0)):
    """Runs the entire segmentation algorithm.
    
    Inputs:
    
      img (ndarray) : a grayscale image to be segmented.
      
      numiter (int) : the number of iterations to compute.
      
      alpha (tuple (int)) : two constants used to calculate 
              geometric annealing temperature.
              
    Outputs:
      
      segmentation (ndarray) : an array of the same shape as 
            img where each entry is an integer indicating the
            class to which that pixel should belong.
    """
    
    # Initialize parameters.
    params = (None, None, None)
    
    # Segment the image.
    for t in range(numiter):
        # Obtain the temperature.
        T_t = (1 + alpha[0])**(alpha[1]*(1 - t/numiter))
        
        # Re-segment.
        segmentation = re_segment(img, t, alpha, params)

        # Sample noise parameters.
        noise_params = sample_noise() # returns a dict

        # Sample MRF parameters.
        MRF_params = sample_MRF() # returns a dict

        # Sample # of classes.
        num_classes = sample_num_classes() # returns an int
        
        # The parameters consist of noise model params, 
        #  MRF model params, and the number of classes.
        params = (noise_params, MRF_params, num_classes)
    
    return segmentation
    

In [13]:
def get_eta(img_c, i, j):
    """
    Gets the class values of the 4 neighbors
    """
    try:
        i1 = img_c[i-1,j]
    except:
        i1 = img_c[i+1,j]
    try:
        i2 = img_c[i,j-1]
    except:
        i2 = img_c[i,j+1]
    try:
        i3 = img_c[i+1,j]
    except:
        i3 = img_c[i-1,j]
    try:
        i4 = img_c[i,j+1]
    except:
        i4 = img_c[i,j-1]
    return np.array([i1,i2,i3,i4])

def V(X, c):
    mask = -((X == c) - 1/2) * 2
    new = np.zeros((X.shape[0]+2, X.shape[1]+2))
    new[0,1:-1] = mask[1,:]
    new[-1,1:-1] = mask[-2,:]
    new[1:-1,0] = mask[:,1]
    new[1:-1,-1] = mask[:,-2]
    new[1:-1, 1:-1] = mask
    v = new[:-2,1:-1] + new[1:-1, :-2] + new[2:,1:-1] + new[1:-1,2:]
    return v/4

In [14]:
def get_n_c(X):
    n_c = [0]*set(X.ravel())
    for x in X.ravel():
        n_c[x] += 1
    return n_c

In [3]:
def re_segment(img, t, alpha, params):
    """
    Returns the segmentation array by computing the probability
    of each pixel being in each class and returning the argmax
    Equation 6 in the paper
    """
    noise_params, MRF_params, num_classes = params
    
    # Obtain the temperature.
    T_t = (1 + alpha[0])**(alpha[1]*(1 - t/numiter))
    
    V = lambda c,η: 1/4*[ np.where(c==t,-1,1) for t in η] 
    
    probs = np.zeros((num_classes, img.shape[0], img.shape[1]))
    
    for c in range(num_classes):
        # Get the parameters for this class.
        mu = noise_params['mu'][c]
        sig = noise_params['sig'][c]
        beta0 = MRF_params['beta0'][c]
        beta1 = MRF_params['beta1'][c]
        
        # Compute the probabilities for this class
        probs[c] = 1/np.sqrt(2*np.pi*T_t*sig**2)*np.exp(
            -1/T_t*(0.5*((img - mu)/sig)**2 
             + (beta0 + beta1*V(c, eta))) # FIXME: what's eta?
        ) # FIXME: define the function V()
    
    
    return np.argmax(probs, axis=0)

In [4]:
def sample_noise():
    """Samples the noise model parameters. Equation 7 in the paper.
    The output is a dictionary with these keys:
    {
     'mu': an array with the mean for each class
     'sig': an array with the st. dev. for each class
    }"""
    
    for c in range(num_classes):
        prob = 1/(sig_c*(2*np.pi*sig_c**2 * T_t)**n_c)
        class_mask = img[np.]
        
    return mu, sig

In [5]:
def sample_MRF(X, params):
    """Samples the MRF model parameters.
    The output is a dictionary with these keys:
    {
     'beta0': an array with beta0 for each class 
     'beta1': an array with beta1 for each class
    }
    """
    noise_params, MRF_params, num_classes = params
    
    
    
    # beta1 is chosen apriori, right above 3.3
    
    
    pass

In [6]:
def sample_num_classes():
    """Return an integer with the number of classes."""
    pass

In [10]:
x = np.array([[1,2,3],[2,2,2]])

In [12]:
set(x.ravel())

{1, 2, 3}

In [15]:
!pwd

/Users/mckayharward/Documents/ACME_files/Math404/mrf-unsupervised-image-segmentation
